In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.pipeline import Pipeline

In [2]:
X = pd.read_csv (r'../data/raw/train_data.csv', header=None)
y = pd.read_csv (r'../data/raw/train_labels.csv', names = ['target'], header=None)

PCA + TSNE

In [4]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

pca_99 = PCA(n_components=0.99)
pca_99_result = pca_99.fit_transform(X)

print(pca_99.n_components_)

2519


In [ ]:
plt.scatter(pca_99_result[:,0], pca_99_result[:,1],c=y.target, alpha=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [11]:
tsne = TSNE(init='pca', n_components=2, learning_rate=100)
tsne_result = tsne.fit_transform(pca_99_result)

/home/michal/anaconda3/envs/CDV/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [ ]:
plt.scatter(tsne_result[:,0], tsne_result[:,1],c=y.target, alpha=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
pdf = pd.DataFrame()

pdf['first_dim'] = pd.Series(tsne_result[:,0])
pdf['second_dim'] = pd.Series(tsne_result[:,1])

plt.figure(figsize=(16,10))
sns.scatterplot(
    x='first_dim', y='second_dim',
    hue=y.target,
    #paleta=sns.color_palette("hls", 10) ,
    data=pdf,
    legend="full",
    #alfa=0.3
)


KernelPCA
gamma deault: 1/columns
best kernel: linear
columns: 90

In [ ]:
clf = Pipeline([
    ("kpca", KernelPCA(n_components=90)),
    ("lgbl", lgb.LGBMClassifier(class_weight="balanced", verbose=1))
])

params = [{
    #"kpca__eigen_solver": ["dense", "arpack"],
    #"kpca__kernel": ["linear", "rbf", "sigmoid"],
    #"kpca__n_components": [70, 90, 100, 1sigmoid10, 130]
    "kpca__gamma": np.linspace(1/10000, 0.03, 3),
}]

grid_search = GridSearchCV(clf, params, cv=3)
grid_search.fit(X,y)

print(grid_search.best_score_)
print(grid_search.best_params_)

In [22]:
kpca = KernelPCA(n_components=90, gamma=None, kernel="linear")
kpca_result = kpca.fit_transform(X, y)

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
kpca_std_result = scaler.fit_transform(kpca_result)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(kpca_std_result, y, test_size=0.2, stratify=y, random_state=2022)
X_train

array([[-1.11991082, -0.684494  ,  1.35635486, ..., -0.19424285,
        -0.56098076, -0.27337674],
       [-0.92425593, -0.97201824, -0.56643876, ..., -0.89483565,
         0.85781199,  1.23305993],
       [ 0.19487417,  1.73973311, -0.20351326, ..., -0.70152106,
        -0.83120052,  1.62899052],
       ...,
       [ 0.84330393,  2.08211302, -0.14192289, ...,  0.18895045,
        -0.01083331, -0.84253365],
       [-0.93957038,  1.93247083,  0.00755928, ...,  0.10534929,
        -0.8100673 ,  0.75138745],
       [ 0.0830381 ,  0.31175602,  0.58974763, ...,  1.26634833,
        -0.18053401,  0.61190851]])

In [46]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

oversampling = RandomOverSampler(random_state=42)
undersampling = RandomUnderSampler(random_state=42, replacement=True)

pipeline = Pipeline(steps=[('o', oversampling), ('u', undersampling)])

X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)
X_resampled

array([[ 0.19487417,  1.73973311, -0.20351326, ..., -0.70152106,
        -0.83120052,  1.62899052],
       [ 0.45514081, -0.13598685,  0.98780612, ..., -1.35835783,
        -0.23677305, -1.80703073],
       [ 0.05932889, -0.95787998,  0.45437466, ...,  1.16603659,
         0.33616222, -2.83223018],
       ...,
       [ 0.88714269,  0.49595305, -0.60587619, ...,  0.04311716,
        -0.18154224,  1.04639135],
       [-1.47425733,  0.84086078, -0.24436884, ...,  1.2193757 ,
         0.83622602, -2.93091305],
       [ 1.14591455, -1.36810659, -0.13973701, ..., -0.71817386,
         0.11754955,  1.51029346]])

In [44]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(criterion='entropy', random_state=1, splitter='best')
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
print('end')

end


In [45]:
f1_score(y_test, y_pred)

0.9242199108469539

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,-91593.433900,96789.870578,-36895.945524,-13438.340570,4446.641134,180.541599,1391.444989,2303.428321,694.920894,-1517.439428,...,304.266926,113.600339,1033.359456,830.801390,-702.619023,-340.348168,444.294740,-1132.661745,-461.007215,-3233.824781
1,184154.264148,4749.553277,1759.891228,38521.039305,-3496.245433,-79.622927,-2583.113559,-2712.715740,-2296.127548,-461.881682,...,-752.904091,2537.344976,-1019.333630,-2550.238524,518.574377,-3078.092073,1119.730756,-452.549802,-205.888447,-1531.714284
2,263109.390985,10679.885565,5933.124977,-35429.371821,7295.375641,157.962689,-1739.946487,1301.481744,4212.887661,-208.019751,...,-734.108837,-1282.443904,-64.863567,1389.229961,573.480029,944.872342,3620.119959,1270.742412,1134.781842,397.470381
3,87336.904757,-27735.864238,49725.460892,-14681.084450,-233.944929,-424.411501,-782.403254,-1661.345018,-1781.862850,-130.470305,...,-1690.177343,-398.898292,772.198900,461.937065,1800.198802,-1130.189027,545.152398,-1939.143021,1234.216482,810.472876
4,-126165.683998,-3049.310040,-50607.767913,26985.201978,-7745.263946,-1.897623,-1202.876071,-961.946665,24.122546,3315.369753,...,1493.057730,-1366.850864,-478.726768,-1633.759057,92.790217,-1719.642380,268.071720,-1257.722082,208.942065,-818.893917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,30852.375296,2443.760681,-37515.920444,288.396969,12464.677853,1655.498087,-748.206815,-642.513156,-1220.942748,1463.666909,...,-1282.544995,-1338.373618,-483.676790,1163.916176,-1106.063253,3581.639425,-940.237229,-3331.808373,-731.392474,778.152787
3746,219757.423663,40321.240020,55605.576151,-12539.525618,-6053.569275,1469.568707,-1387.691304,-1144.604392,1245.224184,1263.831802,...,481.105638,821.957459,5292.403356,491.280360,-2095.989723,-555.094057,-3033.338063,1129.920523,-740.970252,-1129.176380
3747,179459.091091,11374.561213,-15198.363507,-2653.048762,-7248.666273,490.905755,-1834.723328,1339.468223,-332.318377,-2003.026207,...,-1584.020920,164.864685,-19.869329,-1914.183864,601.807872,395.926409,-2324.838996,171.118455,1412.326174,122.648314
3748,-144222.988930,-26324.644198,-40555.204210,-987.763333,9826.795246,-204.449301,-3068.778408,-849.747627,3752.101591,-1669.777305,...,1296.241932,849.101965,2768.577635,3203.240988,730.781056,-204.692443,185.841519,1032.122163,2372.807511,131.957871
